**Nöral Stil Aktarımı**
Nöral stil aktarımı, bir referans görüntünün stilini bir hedefe uygulamaktan oluşur. Stil transferi (style transfer) gibi işlemlerle ünlü ressamların tarzlarıyla yeni resimler çizilmesi ve atın zebraya dönüştürülmesi gibi örnekler yapilabilir. Örnek verecek olursak günümüz mobil uygulamalarından olan  "Prisma" adlı mobil uygulama nöral stil aktarımı tabanlıdır. Uygulama; "Pablo Picasso, Edvard Munch ve Vincent Van Gogh" gibi ünlü sanatçıların sanat stillerini ve diğer sanat eserlerini almak için yapay zeka ve konvolüsyonel sinir ağları adlı bir derin öğrenme yöntemini kullanır ve sadece son resmin oluşturulmasında rehberlik için orijinal resme başvurur.

In [0]:
#Kullanilacak resimler drive üzerinden alınacak. Bu nedenle colab-drive bağlantısı yapılır. 
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [2]:
import keras,os
keras.__version__

'2.1.6'

In [0]:
from keras.preprocessing.image import load_img, img_to_array

In [4]:
# Bu, donusturmek istediğiniz resmin yoludur.
target_image_path = os.path.join("/content/drive", "Colab Notebooks", "neural_style_transfer", "image2.jpg")
print(target_image_path)
# Bu, stil resminin yoludur.
style_reference_image_path = os.path.join("/content/drive", "Colab Notebooks", "neural_style_transfer", "style2.jpg")
print(style_reference_image_path)
# Oluşturulan resmin boyutları
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

/content/drive/Colab Notebooks/neural_style_transfer/portrait.png
/content/drive/Colab Notebooks/neural_style_transfer/popova.jpg


In [0]:
import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    # Sıfır merkezini ortalama piksele göre kaldırın
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB' donusumu yapilir.
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [6]:
from keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))

# Bu yer tutucu, oluşturduğumuz resmi içerecek
combination_image = K.placeholder((1, img_height, img_width, 3))

#3 görüntüyü birleştiriyoruz
input_tensor = K.concatenate([target_image,
                              style_reference_image,
                              combination_image], axis=0)

# VGG19 ağını 3 resim grubumuzla girdi olarak oluşturuyoruz.
# Model, önceden eğitilmiş ImageNet ağırlıklarıyla yüklenir.
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet',
                    include_top=False)
print('Model yüklendi.')

Model yüklendi.


In [0]:
#icerik kaybi methodu
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [0]:
#stil kaybı için gram matris kullanilir
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram


def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [0]:
def total_variation_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [0]:
# Katman adlarını etkinleştirme tensörlerine eşleme
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
# İçerik kaybı için kullanılan katmanın adı
content_layer = 'block5_conv2'
# Stil kaybı için kullanılan katmanların adı
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
# Kayıp bileşenlerinin ağırlıklı ortalama ağırlıkları
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

#Tum bileşenleri bir `loss` değişkenine ekleyerek kayıp tanımlanir
loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(target_image_features,
                                      combination_features)
#her hedef katman için bir stil kaybi bileseni ekler.
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl
#toplam varyasyon kaybini ekler
loss += total_variation_weight * total_variation_loss(combination_image)

In [0]:
# Oluşturulan görüntünün gradyanlarını kaybı
grads = K.gradients(loss, combination_image)[0]

# Mevcut kayıp ve akım gradyanlarının değerlerini almak için fonksiyon
fetch_loss_and_grads = K.function([combination_image], [loss, grads])


class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        #fetch_loss_and_grads kayiplari geri almanizi saglayacak sekilde arar
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [54]:
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time

result_prefix = 'style_transfer_result'
iterations = 20

# Nöral stil kaybını en aza indirmek için oluşturulan görüntünün pikselleri üzerinde scipy tabanlı optimizasyonu (L-BFGS) çalıştırılır
# Bu bizim başlangıç durumumuz: hedef imaj.
x = preprocess_image(target_image_path)
#görüntüyü düzlestirir
#Cunku Scipy.optimize.fmin_l_bfgs_b` nin sadece düz vektörleri işleyebilir
x = x.flatten()
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,
                                     fprime=evaluator.grads, maxfun=20)
    #noral stil kaybini en z-aza indirmek için olusan piksellerin üzerinde fmin_l_bfgs_b calistirilir.
    print('Current loss value:', min_val)
    # Mevcut oluşturulan resmi kaydet
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    imsave(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 11603254000.0
Image saved as style_transfer_result_at_iteration_0.png
Iteration 0 completed in 267s
Start of iteration 1
Current loss value: 4385868300.0
Image saved as style_transfer_result_at_iteration_1.png
Iteration 1 completed in 264s
Start of iteration 2
Current loss value: 2734194200.0
Image saved as style_transfer_result_at_iteration_2.png
Iteration 2 completed in 265s
Start of iteration 3
Current loss value: 1983815200.0
Image saved as style_transfer_result_at_iteration_3.png
Iteration 3 completed in 267s
Start of iteration 4
Current loss value: 1609425900.0
Image saved as style_transfer_result_at_iteration_4.png
Iteration 4 completed in 263s
Start of iteration 5
Current loss value: 1362422100.0
Image saved as style_transfer_result_at_iteration_5.png
Iteration 5 completed in 275s
Start of iteration 6
Current loss value: 1182633700.0
Image saved as style_transfer_result_at_iteration_6.png
Iteration 6 completed in 259s
Start of iteration 

In [0]:
from matplotlib import pyplot as plt

#İçerik resmi
plt.imshow(load_img(target_image_path, target_size=(img_height, img_width)))
plt.figure()

# Stil görüntü
plt.imshow(load_img(style_reference_image_path, target_size=(img_height, img_width)))
plt.figure()

# Oluşan görüntü
plt.imshow(img)
plt.show()